In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
from scipy.spatial.distance import euclidean
from tqdm import tqdm
from conehead.geometry import (
    global_to_beam, line_block_plane_collision, line_calc_limit_plane_collision
)
from conehead.kernel import PolyenergeticKernel
# from conehead.dda_3d_c import dda_3d_c
import conehead.nist
from conehead.source import Source
from conehead.block import Block
from conehead.phantom import SimplePhantom
from conehead.conehead import Conehead

ModuleNotFoundError: No module named 'conehead.geometry'; 'conehead' is not a package

In [ ]:
# Choose source
source = Source("varian_clinac_6MV")
source.gantry(0)
source.collimator(0)

# Create 10 cm x 10 cm collimator opening
block = Block(source.rotation)
block.set_square(10)

# Simple phantom
phantom = SimplePhantom()

# Calculation settings
settings = {
    'stepSize': 0.1,  # Stepsize when raytracing effective depth
    'sPri': 1.0,  # Primary source strength (photons/mm^2)
    'softRatio': 0.0025,  # mm^-1
    'softLimit': 20,  # cm
    'eLow': 0.01,  # MeV
    'eHigh': 7.0,  # MeV
    'eNum': 500,  # Spectrum samples
}

In [3]:
# Transform phantom to beam coords
print("Transforming phantom to beam coords...")
phantom_beam = np.zeros_like(phantom.positions)
_, xlen, ylen, zlen = phantom_beam.shape
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            phantom_beam[:, x, y, z] = global_to_beam(
                phantom.positions[:, x, y, z],
                source.position,
                source.rotation
            )

print("Interpolating phantom densities...")
phantom_densities_interp = RegularGridInterpolator(
    (phantom_beam[0, :, 0, 0],
     phantom_beam[1, 0, :, 0],
     phantom_beam[2, 0, 0, :]),
    phantom.densities,
    method='nearest',
    bounds_error=False,
    fill_value=0
)

# Create dose grid (just the same size as the phantom for now)
dose_grid_positions = np.copy(phantom_beam)
dose_grid_dim = np.array([2, 2, 2], dtype=np.float64)  # cm

# Perform hit testing to find which dose grid voxels are in the beam
print("Performing hit-testing of dose grid voxels...")
_, xlen, ylen, zlen = dose_grid_positions.shape
dose_grid_blocked = np.zeros((xlen, ylen, zlen))
dose_grid_OAD = np.zeros((xlen, ylen, zlen))
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            voxel = dose_grid_positions[:, x, y, z]
            psi = line_block_plane_collision(voxel)
            dose_grid_blocked[x, y, z] = (
                block.block_values_interp([psi[0], psi[1]])
            )
            # Save off-axis distance (at iso plane) for later
            dose_grid_OAD[x, y, z] = (
                euclidean(np.array([0, 0, source.SAD]), psi)
            )

# Calculate effective depths of dose grid voxels
print("Calculating effective depths of dose grid voxels...")
dose_grid_d_eff = np.zeros_like(dose_grid_blocked)
xlen, ylen, zlen = dose_grid_d_eff.shape
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            voxel = dose_grid_positions[:, x, y, z]
            psi = line_calc_limit_plane_collision(voxel)
            dist = np.sqrt(np.sum(np.power(voxel - psi, 2)))
            num_steps = np.floor(dist / settings['stepSize'])
            xcoords = np.linspace(voxel[0], psi[0], num_steps)
            ycoords = np.linspace(voxel[1], psi[1], num_steps)
            zcoords = np.linspace(voxel[2], psi[2], num_steps)
            dose_grid_d_eff[x, y, z] = np.sum(
                phantom_densities_interp(
                    np.dstack((xcoords, ycoords, zcoords))
                ) * settings['stepSize']
            )

# Calculate photon fluence at dose grid voxels
print("Calculating fluence...")
dose_grid_fluence = np.zeros_like(dose_grid_blocked)
xlen, ylen, zlen = dose_grid_fluence.shape
dose_grid_fluence = (
    settings['sPri'] * -source.SAD /
    dose_grid_positions[2, :, :, :] *
    dose_grid_blocked
)

# Calculate beam softening factor for dose grid voxels
print("Calculating beam softening factor...")
f_soften = np.ones_like(dose_grid_OAD)
f_soften[dose_grid_OAD < settings['softLimit']] = 1 / (
    1 - settings['softRatio'] *
    dose_grid_OAD[dose_grid_OAD < settings['softLimit']]
)

# Calculate TERMA of dose grid voxels
print("Calculating TERMA...")
E = np.linspace(settings['eLow'], settings['eHigh'], settings['eNum'])
spectrum_weights = source.weights(E)
mu_water = conehead.nist.mu_water(E)
dose_grid_terma = np.zeros_like(dose_grid_blocked)
xlen, ylen, zlen = dose_grid_terma.shape
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            dose_grid_terma[x, y, z] = (
                np.sum(
                    spectrum_weights *
                    dose_grid_fluence[x, y, z] *
                    np.exp(
                        -mu_water * f_soften[x, y, z] *
                        dose_grid_d_eff[x, y, z]
                    ) * E * mu_water
                )
            )


 14%|█▍        | 3/21 [00:00<00:00, 26.97it/s]

Transforming phantom to beam coords...


 10%|▉         | 2/21 [00:00<00:01, 12.19it/s]

Interpolating phantom densities...
Performing hit-testing of dose grid voxels...


  0%|          | 0/21 [00:00<?, ?it/s]/home/sam/projects/conehead/env/lib/python3.6/site-packages/ipykernel_launcher.py:58: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
/home/sam/projects/conehead/env/lib/python3.6/site-packages/ipykernel_launcher.py:59: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
/home/sam/projects/conehead/env/lib/python3.6/site-packages/ipykernel_launcher.py:60: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  5%|▍         | 1/21 [00:00<00:03,  5.78it/s]

Calculating effective depths of dose grid voxels...


 43%|████▎     | 9/21 [00:00<00:00, 88.26it/s]

Calculating fluence...
Calculating beam softening factor...
Calculating TERMA...


100%|██████████| 21/21 [00:00<00:00, 88.52it/s]


In [49]:
# Calculate dose of dose grid voxels
print("Convolving kernel...")
kernel = PolyenergeticKernel()
dose_grid_dose = np.zeros_like(dose_grid_terma)
phis = [p for p in kernel.cumulative.keys() if p != "radii"]
thetas = np.linspace(0, 360, 6, endpoint=False)

Convolving kernel...


In [50]:
def convolve_python(dose_grid_terma, dose_grid_dose, dose_grid_dim, thetas, phis, kernel):

    xlen, ylen, zlen = dose_grid_terma.shape
    
    for x in tqdm(range(xlen)):
        for y in range(ylen):
            for z in range(zlen):
                T = dose_grid_terma[x, y, z]
                if T:
                    for theta in thetas:
                        for phi in phis:

                            # Raytracing
                            phi_rad = float(phi) * np.pi / 180
                            theta_rad = theta * np.pi / 180
                            direction = np.array([
                                np.cos(theta_rad) * np.sin(phi_rad),
                                np.sin(theta_rad) * np.sin(phi_rad),
                                np.cos(phi_rad)
                            ], dtype=np.float64)
                            direction /= np.sum(direction**2)  # Normalise
                            direction = np.around(  # discretise
                                direction,
                                decimals=6
                            )
                            intersections, voxels = dda_3d_c(
                                direction,
                                np.array(
                                    dose_grid_terma.shape,
                                    dtype=np.int32
                                ),
                                np.array([x, y, z], dtype=np.int32),
                                dose_grid_dim
                            )

                            intersections = np.array(
                                [int(x) for x in (intersections * 100 - 50)]
                            )
                            intersections = np.absolute(intersections)

                            for e, _ in enumerate(intersections):
                                v = voxels[e]
                                if e == 0:
                                    k = kernel.cumulative[phi][
                                        intersections[e]
                                    ]
                                else:
                                    k = kernel.cumulative[phi][
                                        intersections[e]
                                    ] - kernel.cumulative[phi][
                                        intersections[e - 1]
                                    ]
                                dose_grid_dose[v[0], v[1], v[2]] += T * k

In [53]:
%%time
convolve_python(dose_grid_terma, dose_grid_dose, dose_grid_dim, thetas, phis, kernel)

100%|██████████| 21/21 [00:26<00:00,  1.27s/it]

CPU times: user 26.6 s, sys: 220 ms, total: 26.8 s
Wall time: 26.8 s


In [4]:
load_ext Cython

In [5]:
# Calculate dose of dose grid voxels
print("Convolving kernel...")
kernel = PolyenergeticKernel()
dose_grid_dose = np.zeros_like(dose_grid_terma, dtype=np.float64)
phis = np.array([p for p in kernel.cumulative.keys() if p != "radii"], dtype=np.float64)
thetas = np.linspace(0, 360, 6, endpoint=False, dtype=np.float64)
dose_grid_dim = np.array([2, 2, 2], dtype=np.float64)

Convolving kernel...


In [6]:
%%cython -a

import numpy as np
cimport cython
cimport numpy as cnp
from libc.math cimport sin, cos, ceil
from conehead.dda_3d_c cimport dda_3d_c, result
from conehead.vector cimport vector, vector_init, vector_append, vector_get, vector_set, vector_free

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True)
def convolve_c_naive(cnp.float64_t[:,:,:] dose_grid_terma, cnp.float64_t[:,:,:] dose_grid_dose, cnp.float64_t[:] dose_grid_dim, cnp.float64_t[:] thetas, cnp.float64_t[:] phis, kernel):

    cdef cnp.int32_t xlen = dose_grid_terma.shape[0]
    cdef cnp.int32_t ylen = dose_grid_terma.shape[1]
    cdef cnp.int32_t zlen = dose_grid_terma.shape[2]
    cdef cnp.int32_t dose_grid_shape[3]
    dose_grid_shape = [
        xlen, ylen, zlen
    ]
    
    cdef cnp.float64_t T, N, N_inv, theta_rad, phi_rad, c_t, s_t, c_p, s_p
    cdef cnp.float64_t pi = 3.14159265359
    cdef cnp.int32_t num_thetas = thetas.shape[0]
    cdef cnp.int32_t num_phis = phis.shape[0]
    cdef cnp.int32_t x, y, z, i, j

    cdef cnp.int32_t current_voxel[3]
    cdef cnp.float64_t direction[3]
    cdef cnp.float64_t dimensions[3] 
    dimensions = [
        dose_grid_dim[0], 
        dose_grid_dim[1],
        dose_grid_dim[2]
    ]
    
    cdef result r
    
    cdef cnp.float64_t[:] intersections
    cdef cnp.float64_t* voxels
    
    for x in range(xlen):
        for y in range(ylen):
            for z in range(zlen):

                T = dose_grid_terma[x, y, z]
                if T:

                    for i in range(num_thetas):
                        for j in range(num_phis):
                            
                            current_voxel = [x, y, z]

                            # Calculate direction vector
                            theta_rad = thetas[i] * pi / 180.0
                            phi_rad = phis[j] * pi / 180.0
                            c_t = cos(theta_rad)
                            s_t = sin(theta_rad)
                            c_p = cos(phi_rad)
                            s_p = sin(phi_rad)
                            direction = [
                                c_t * s_p,
                                s_t * s_p,
                                c_p
                            ]
                            N = direction[0] * direction[0] + direction[1] * direction[1] + direction[2] * direction[2]
                            N_inv = 1.0 / N
                            direction = [  # Normalise
                                direction[0] * N_inv,
                                direction[1] * N_inv,
                                direction[2] * N_inv
                            ]
                            direction = [  # Discretise
                                ceil(direction[0] * 100000) * 0.00001,
                                ceil(direction[1] * 100000) * 0.00001,
                                ceil(direction[2] * 100000) * 0.00001
                            ]
                                                                   
                            # Raytracing
                            intersections, voxels = dda_3d_c(
                                direction,
                                dose_grid_shape,
                                current_voxel,
                                dimensions
                            )

                            intersections = np.array(
                                [int(x) for x in (intersections * 100 - 50)]
                            )
                            intersections = np.absolute(intersections)

                            phi_str = "{:.2f}".format(phis[j])
                            
                            for e, _ in enumerate(intersections):
                                v = voxels[e]
                                if e == 0:
                                    k = kernel.cumulative[phi_str][
                                        intersections[e]
                                    ]
                                else:
                                    k = kernel.cumulative[phi_str][
                                        intersections[e]
                                    ] - kernel.cumulative[phi_str][
                                        intersections[e - 1]
                                    ]
                                dose_grid_dose[v[0], v[1], v[2]] += T * k


Error compiling Cython file:
------------------------------------------------------------
...
                                ceil(direction[1] * 100000) * 0.00001,
                                ceil(direction[2] * 100000) * 0.00001
                            ]
                                                                   
                            # Raytracing
                            intersections, voxels = dda_3d_c(
                           ^
------------------------------------------------------------

/home/sam/.cache/ipython/cython/_cython_magic_e59e7f63ce14d0fd550941aa02505fa7.pyx:77:28: Cannot convert Python object to 'float64_t *'

Error compiling Cython file:
------------------------------------------------------------
...
                                current_voxel,
                                dimensions
                            )

                            intersections = np.array(
                                [int(x) for x in (intersections * 

TypeError: object of type 'NoneType' has no len()

In [55]:
%%time
convolve_c_naive(dose_grid_terma, dose_grid_dose, dose_grid_dim, thetas, phis, kernel)

CPU times: user 23.9 s, sys: 0 ns, total: 23.9 s
Wall time: 23.9 s


In [70]:
type(np.cos(0.5))

numpy.float64

In [71]:
thetas.dtype

dtype('float64')